In [17]:
# from convert_json_csv import collet_convert_datas
# collet_convert_datas()

In [18]:
from db_manager import create_database, insert_data
import pandas as pd

create_database("assets/database/database_teste.db")
df = pd.read_csv("assets/database/users_datas.csv")

for _, row in df.iterrows():
    # Preparar a query de inserção
    insert_data("assets/database/database_teste.db", row.to_dict())

Banco de dados 'assets/database/database_teste.db' criado com sucesso com a tabela 'users'.
Dados inseridos com sucesso na tabela 'users'.
Dados inseridos com sucesso na tabela 'users'.


In [19]:
import pandas as pd
import numpy as np
import sqlite3
import pandas as pd
import numpy as np



# Função para calcular os pontos
def calcular_pontos(df):
    if len(df) < 2:
        print("Não há registros suficientes para calcular os pontos.")
        return df

    # Obter os dois registros mais recentes
    registro_atual = df.iloc[0]
    registro_anterior = df.iloc[-1]

    # Calcular pontosXp (usando log10)
    diferenca_xp = registro_atual["totalXp"] - registro_anterior["totalXp"]
    if diferenca_xp > 0:
        pontos_xp = int(10 * np.log10(diferenca_xp))  # Log na base 10
    else:
        pontos_xp = 0  # Se a diferença for <= 0, pontosXp = 0
    df.at[df.index[0], "pontosXp"] = pontos_xp

    # Calcular pontosOfensiva
    diferenca_streak = registro_atual["streak"] - registro_anterior["streak"]
    print(registro_anterior['pontosOfensiva'])
    if registro_anterior['pontosOfensiva']:
        diferenca_streak = + diferenca_streak + (registro_anterior['pontosOfensiva']/5)
    pontos_streak = max(0, diferenca_streak) * 5
    df.at[df.index[0], "pontosOfensiva"] = pontos_streak

    # Calcular pontosTotal
    pontos_total = pontos_xp + pontos_streak
    df.at[df.index[0], "pontosTotal"] = pontos_total
    display(df)
    return df

# Função para atualizar o registro no banco de dados
def atualizar_registro_no_banco(cursor, id_register, pontos_xp, pontos_streak, pontos_total):
    print(f"ID do registro: {id_register}")
    print(f"pontos_xp: {pontos_xp}")
    query = f"""
    UPDATE users
    SET pontosXp = {pontos_xp}, pontosOfensiva = {pontos_streak}, pontosTotal = {pontos_total}
    WHERE idRegister = {id_register}
    """
    cursor.execute(query)
    print(f"Registro {id_register} atualizado no banco de dados.")



print("Processo concluído.")

Processo concluído.


In [20]:
import sqlite3
import csv
import pandas as pd

# Conectar ao banco de dados SQLite
conn = sqlite3.connect("assets/database/database_teste.db")
cursor = conn.cursor()

# Lista para armazenar todos os resultados
all_results = []

# Ler o arquivo CSV de usuários
with open('assets/database/users.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        user_id = row['userId']
        # Executar a consulta SQL
        query = f"SELECT * FROM users WHERE id = ? ORDER BY idRegister DESC LIMIT 2"
        cursor.execute(query, (user_id,))
        results = cursor.fetchall()

        column_names = [
            "idRegister","id", "name", "username", "totalXp", "streak", "creationDate", 
            "hasPlus", "fromLanguage", "learningLanguage", "subscriberLevel", "current_time", "pontosXp", "pontosOfensiva", "pontosTotal"
        ]
        # Adicionar os resultados à lista
        df_results = pd.DataFrame(results, columns=column_names)
        display(df_results)
        # Conectar ao banco de dados
        conn = sqlite3.connect("assets/database/database_teste.db")
        cursor = conn.cursor()
        df = calcular_pontos(df_results)

        # Atualizar o registro mais recente no banco de dados
        if len(df) >= 2:
            registro_atual = df.iloc[0]
            id_register = registro_atual["idRegister"]
            pontos_xp = registro_atual["pontosXp"]
            pontos_streak = registro_atual["pontosOfensiva"]
            pontos_total = registro_atual["pontosTotal"]


            # Atualizar o banco de dados
            atualizar_registro_no_banco(cursor, id_register, pontos_xp, pontos_streak, pontos_total)
        all_results.extend(results)
        conn.commit()


,idRegister,id,name,username,totalXp,streak,creationDate,hasPlus,fromLanguage,learningLanguage,subscriberLevel,current_time,pontosXp,pontosOfensiva,pontosTotal
0,9,894236585,Tamara Oliveira,tamaravoliveira,91208,0,1645052528,1,pt,en,GOLD,2025-02-23 22:33:40,NaN,NaN,NaN
1,7,894236585,Tamara Oliveira,tamaravoliveira,91208,847,1645052528,1,pt,en,GOLD,2025-02-22 22:33:40,0.0,10.0,10.0


10.0


,idRegister,id,name,username,totalXp,streak,creationDate,hasPlus,fromLanguage,learningLanguage,subscriberLevel,current_time,pontosXp,pontosOfensiva,pontosTotal
0,9,894236585,Tamara Oliveira,tamaravoliveira,91208,0,1645052528,1,pt,en,GOLD,2025-02-23 22:33:40,0.0,0.0,0.0
1,7,894236585,Tamara Oliveira,tamaravoliveira,91208,847,1645052528,1,pt,en,GOLD,2025-02-22 22:33:40,0.0,10.0,10.0


ID do registro: 9
pontos_xp: 0.0
Registro 9 atualizado no banco de dados.


,idRegister,id,name,username,totalXp,streak,creationDate,hasPlus,fromLanguage,learningLanguage,subscriberLevel,current_time,pontosXp,pontosOfensiva,pontosTotal
0,10,280512491,Lucas Tamura,lucas.tamura,15120,170,1494871701,1,pt,en,GOLD,2025-02-23 22:33:41,NaN,NaN,NaN
1,8,280512491,Lucas Tamura,lucas.tamura,15120,170,1494871701,1,pt,en,GOLD,2025-02-22 22:33:41,0.0,15.0,15.0


15.0


,idRegister,id,name,username,totalXp,streak,creationDate,hasPlus,fromLanguage,learningLanguage,subscriberLevel,current_time,pontosXp,pontosOfensiva,pontosTotal
0,10,280512491,Lucas Tamura,lucas.tamura,15120,170,1494871701,1,pt,en,GOLD,2025-02-23 22:33:41,0.0,15.0,15.0
1,8,280512491,Lucas Tamura,lucas.tamura,15120,170,1494871701,1,pt,en,GOLD,2025-02-22 22:33:41,0.0,15.0,15.0


ID do registro: 10
pontos_xp: 0.0
Registro 10 atualizado no banco de dados.
